In [1]:
from MIOFlow.models import make_model
import torch

In [2]:
batch_size = 4
feature_dim = 5
output_dim = 5

x = torch.randn(batch_size, feature_dim)
# m0 = torch.ones(batch_size, 1)
# xm = torch.cat([x, m0], dim=1)
t = torch.linspace(0, 1, 7)

model = make_model(feature_dims=feature_dim, which='ode_growth_rate')

In [3]:
xt, mt = model(x, t)

In [4]:
xt.shape, mt.shape

(torch.Size([4, 5]), torch.Size([4]))

In [5]:
xtseq, mtseq = model(x, t, return_whole_sequence=True)

In [6]:
xtseq.shape

torch.Size([7, 4, 5])

In [7]:
mtseq.shape

torch.Size([7, 4])

In [8]:
torch.nn.functional.softmax(mt, dim=-1) * mt.shape[-1]

tensor([0.9534, 0.8871, 1.0050, 1.1545], grad_fn=<MulBackward0>)

In [9]:
torch.exp(mt)/torch.exp(mt).sum(dim=0)

tensor([0.2384, 0.2218, 0.2512, 0.2886], grad_fn=<DivBackward0>)

In [10]:
torch.nn.functional.softmax(mtseq, dim=-1) * mtseq.shape[-1]

tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [0.9904, 0.9801, 1.0045, 1.0250],
        [0.9821, 0.9601, 1.0076, 1.0502],
        [0.9745, 0.9405, 1.0095, 1.0756],
        [0.9676, 0.9214, 1.0096, 1.1014],
        [0.9609, 0.9033, 1.0080, 1.1278],
        [0.9534, 0.8871, 1.0050, 1.1545]], grad_fn=<MulBackward0>)

In [11]:
xtseq.shape

torch.Size([7, 4, 5])

In [12]:
mtseq.unsqueeze(2).shape

torch.Size([7, 4, 1])

In [13]:
xmseq = torch.cat([xtseq, mtseq.unsqueeze(2)], dim=-1)
xmseq.shape

torch.Size([7, 4, 6])

In [14]:
dxdts = torch.stack([model.func(t[i], xmseq[i]) for i in range(len(t))])

In [15]:
dxdts.shape

torch.Size([7, 4, 6])

In [16]:
mt.unsqueeze(1).shape

torch.Size([4, 1])

In [17]:
model.func

ToyODE(
  (seq): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [18]:
xt.shape

torch.Size([4, 5])

In [19]:
mt.shape

torch.Size([4])

In [20]:
xm = torch.cat([xt, mt.unsqueeze(-1)], dim=-1)
dxdt = model.func(t[-1], xm)

In [21]:
dxdt[...,:-1].shape

torch.Size([4, 5])

In [22]:
mtseq.shape

torch.Size([7, 4])

In [23]:
xtseq.shape

torch.Size([7, 4, 5])

In [24]:
t.shape

torch.Size([7])

In [25]:
t[[0,1,2],...]

tensor([0.0000, 0.1667, 0.3333])

In [26]:
mt.shape

torch.Size([4])

In [27]:
(dxdt * mt.unsqueeze(-1)).shape

torch.Size([4, 6])

In [28]:
dxdts.shape

torch.Size([7, 4, 6])

In [29]:
mtseq.shape

torch.Size([7, 4])

In [30]:
(dxdts * mtseq.unsqueeze(-1)).shape

torch.Size([7, 4, 6])

In [31]:
torch.nn.functional.softmax(mtseq, dim=-1).shape

torch.Size([7, 4])

In [32]:
mtseq.shape[-1]

4

In [33]:
torch.nn.functional.softmax(mt, dim=-1).shape

torch.Size([4])

In [34]:
mt.shape[-1]

4

In [35]:
def func_end_0(t, x):
    xm = model.func(t, x)
# mt[[0,2,4,6,8]] = 0.
model.func(t[-1], xm).shape

torch.Size([4, 6])

In [36]:
xm[...,-1].shape

torch.Size([4])

In [37]:
xm.shape

torch.Size([4, 6])

In [38]:
xmseq.shape

torch.Size([7, 4, 6])

In [39]:
xm[[0,2],-1] = 0.

In [40]:
xm

tensor([[-0.9419,  0.3272, -1.6806,  0.6901, -0.2276,  0.0000],
        [-1.6372, -0.4944, -0.3773, -0.5397, -1.2240, -0.1971],
        [ 0.1125, -0.3432, -0.3555,  0.1644,  0.2746,  0.0000],
        [ 0.0769, -1.0837, -1.7402,  1.4943, -1.3763,  0.0664]],
       grad_fn=<CopySlices>)

In [41]:
dxmdt = model.func(t[-1], xm)

In [42]:
dxmdt

tensor([[ 0.1445, -0.6833, -0.2113,  0.2419, -0.0974, -0.1340],
        [ 0.0669, -0.6117, -0.3840,  0.0490, -0.1105, -0.1943],
        [ 0.0806, -0.2887, -0.1269,  0.2214,  0.0333, -0.1062],
        [ 0.0952, -0.9277, -0.1679,  0.5920, -0.3656,  0.0480]],
       grad_fn=<AddmmBackward0>)